In [1]:
# from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
# from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
# from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np
import torch
import torch.nn as nn
import os
import sys
from gradcam import GradCAM
# Back to root
sys.path.append('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

from src.models.sequence.visualizer import CNNModel


In [ ]:
from omegaconf import DictConfig

args = DictConfig({
    'clean_data': False,
    'hidden_dim': 128,
    'cls_expanded_simplex': False,
    'mode': 'dirichlet',
    'num_cnn_stacks': 1,
    'dropout': 0.0,
    'cls_free_guidance': False,
    'alpha_max': 8,
    'fix_alpha': 1e6,
    'alpha_scale': 2,
    'prior_pseudocount': 2
})


In [ ]:
model = CNNModel(args, alphabet_size=5, num_cls=0, pretrain=False, use_outlinear=True, length=300, for_representation=True)
checkPointPath = "../outputs/2024-05-17/14-12-10-430866/checkpoints/val/f1_macro.ckpt"
state_dict = torch.load(checkPointPath)
state_dict = state_dict['state_dict']
state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}



print(state_dict.keys())
incompatible_keys = model.load_state_dict(state_dict=state_dict, strict=False)
# 检查missing_keys和unexpected_keys
if incompatible_keys.missing_keys:
    print('Missing keys:', incompatible_keys.missing_keys)
if incompatible_keys.unexpected_keys:
    print('Unexpected keys:', incompatible_keys.unexpected_keys)
# print(model)
target_layer = model.out_linear

In [ ]:
model

In [ ]:
DNAdict = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4}

# YAL063C_FLO9_27164
DNAstring = """
GATGGGCGGGGGGGGGGGTCTCCCCGCCACGGGAGGTCTAGAGAGCAGCGGCGGCGGGGCGGGGCGGAGTGCAGAGGCGCCGCCGGCGGGGCGGCTTCGGGAGGGCGCGGCCCCTTTAAGACGCCCCGCCGGCCCCGCCCCCGAGCCCCGCCTCCCGCCGCCCACGTGACCCCGGTCTTGTGACTGGGCCCGGGAGGGCGGGGGAAGCCCGCGGCTCGCGCCCGCCCCGCCCCGCCCCGCGTCTGCCTCAGAGGGGCCCGAGCCACCCGGTCCGCCGCGTCCCCGCCGCCGCCGCCGCGT
"""

def covert_seq_to_np(seq):
    return np.array([DNAdict[x] for x in seq])

DNAstring = DNAstring.replace("\n", "")
seq = covert_seq_to_np(DNAstring)
seq_tensor = torch.tensor(seq).unsqueeze(0)
seq_tensor = seq_tensor.long()
print(seq_tensor.shape)
print(nn.functional.one_hot(seq_tensor, num_classes=5))

In [ ]:
net = GradCAM(model=model, target_layer=target_layer)
output = net(seq_tensor)
print(output)

In [ ]:
import scipy.io as scio
input_tensor = seq_tensor.numpy().squeeze()

scio.savemat('Draw.mat', {'cam': output, 'input': input_tensor})

In [ ]:
mat_file = scio.loadmat('Draw.mat')
# 打印加载的数据内容
for key in mat_file:
    print('%s: %s' % (key, mat_file[key].__class__))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def target_category_loss(x, category_index, nb_classes):
    return torch.mul(x, F.one_hot(category_index, nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # Utility function to normalize a tensor by its L2 norm
    return x / (torch.sqrt(torch.mean(torch.square(x))) + 1e-5)

def resize_1d(array, shape):
    res = np.zeros(shape)
    if array.shape[0] >= shape:
        ratio = array.shape[0] / shape
        for i in range(array.shape[0]):
            res[int(i / ratio)] += array[i] * (1 - (i / ratio - int(i / ratio)))
            if int(i / ratio) != shape - 1:
                res[int(i / ratio) + 1] += array[i] * (i / ratio - int(i / ratio))
            else:
                res[int(i / ratio)] += array[i] * (i / ratio - int(i / ratio))
        res = res[::-1]
        array = array[::-1]
        for i in range(array.shape[0]):
            res[int(i / ratio)] += array[i] * (1 - (i / ratio - int(i / ratio)))
            if int(i / ratio) != shape - 1:
                res[int(i / ratio) + 1] += array[i] * (i / ratio - int(i / ratio))
            else:
                res[int(i / ratio)] += array[i] * (i / ratio - int(i / ratio))
        res = res[::-1] / (2 * ratio)
        array = array[::-1]
    else:
        ratio = shape / array.shape[0]
        left = 0
        right = 1
        for i in range(shape):
            if left < int(i / ratio):
                left += 1
                right += 1
            if right > array.shape[0] - 1:
                res[i] += array[left]
            else:
                res[i] += array[right] * (i - left * ratio) / ratio + array[left] * (right * ratio - i) / ratio
        res = res[::-1]
        array = array[::-1]
        left = 0
        right = 1
        for i in range(shape):
            if left < int(i / ratio):
                left += 1
                right += 1
            if right > array.shape[0] - 1:
                res[i] += array[left]
            else:
                res[i] += array[right] * (i - left * ratio) / ratio + array[left] * (right * ratio - i) / ratio
        res = res[::-1] / 2
        array = array[::-1]
    return res

class ActivationsAndGradients:
    """ Class for extracting activations and registering gradients from targeted intermediate layers """

    def __init__(self, model, target_layer):
        self.model = model
        self.gradients = []
        self.activations = []

        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations.append(output)

    def save_gradient(self, module, grad_input, grad_output):
        # Gradients are computed in reverse order
        self.gradients = [grad_output[0]] + self.gradients

    def __call__(self, x):
        self.gradients = []
        self.activations = []
        return self.model(x)

class BaseCAM:
    def __init__(self, model, target_layer, use_cuda=False):
        self.model = model.eval()
        self.target_layer = target_layer
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.activations_and_grads = ActivationsAndGradients(self.model, target_layer)

    def forward(self, input_img):
        return self.model(input_img)

    def get_cam_weights(self, input_tensor, target_category, activations, grads):
        raise Exception("Not Implemented")

    def get_loss(self, output, target_category):
        return output[:,target_category]

    def __call__(self, input_tensor, target_category=None):
        if self.cuda:
            input_tensor = input_tensor.cuda()

        output = self.activations_and_grads(input_tensor)[0].squeeze(1)

        if target_category is None:
            target_category = np.argmax(output.cpu().data.numpy(), axis=-1)

        self.model.zero_grad()

        loss = self.get_loss(output, target_category)
        loss.backward(retain_graph=True)

        activations = self.activations_and_grads.activations[-1].cpu().data.numpy()[0, :]
        grads = self.activations_and_grads.gradients[-1].cpu().data.numpy()[0, :]
        weights = self.get_cam_weights(input_tensor, target_category, activations, grads)
        cam = np.zeros(activations.shape[1:], dtype=np.float32)
        for i, w in enumerate(weights):
            cam += w * activations[i, :]
        print(cam.shape)
        cam = resize_1d(cam, input_tensor.shape[1])
        cam = np.maximum(cam, 0)
        heatmap = (cam - np.min(cam)) / (np.max(cam) - np.min(cam) + 1e-10)
        return heatmap

class GradCAM(BaseCAM):
    def __init__(self, model, target_layer, use_cuda=False):
        super(GradCAM, self).__init__(model, target_layer, use_cuda)

    def get_cam_weights(self, input_tensor, target_category, activations, grads):
        return np.mean(grads, axis=1)

target_layer = model.convs[1]
net = GradCAM(model, target_layer)
# from settest import Test
# input_tensor = Test.Data[100:101, :]
input_tensor = seq_tensor
# input_tensor = torch.tensor(input_tensor, dtype=torch.float32)
# #plt.figure(figsize=(5, 1))
output = net(input_tensor)
import scipy.io as scio
input_tensor = input_tensor.numpy().squeeze()
dataNew = "./new.mat"
scio.savemat(dataNew, mdict={'cam': output, 'data': input_tensor})

In [ ]:
model(input_tensor)[0].shape

In [ ]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

# Load the data from the .mat file
data = sio.loadmat('new.mat')

# Extract the CAM and DNA data
cam = data['cam'].squeeze()  # Ensure CAM is 1D
dna_sequence = data['data']

# Ensure the data is in the correct shape
print('CAM shape:', cam.shape)
print('DNA sequence shape:', dna_sequence.shape)

# Plot the DNA sequence as a heatmap if it's 2D
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.title('DNA Sequence Heatmap')
if len(dna_sequence.shape) == 2:
    plt.imshow(dna_sequence, aspect='auto', cmap='Greys', norm=plt.cm.colors.PowerNorm(gamma=0.5, vmin=np.min(dna_sequence[dna_sequence > 0]),
                                                                                         vmax=np.max(dna_sequence)))
    plt.colorbar(label='Intensity')
    plt.xlabel('Sequence Position')
    plt.ylabel('Feature Index')
else:
    plt.plot(dna_sequence)
    plt.xlabel('Sequence Position')
    plt.ylabel('Intensity')

# Plot the CAM result as a line plot
plt.subplot(1, 2, 2)
plt.title('Class Activation Map (CAM)')
plt.plot(cam)
plt.xlabel('Sequence Position')
plt.ylabel('CAM Intensity')

# Shift the y-axis for the CAM plot
plt.gca().set_ylim(0.3, plt.gca().get_ylim()[1])

plt.tight_layout()
plt.show()


In [ ]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

# Load the data from the .mat file
data = sio.loadmat('new.mat')

# Extract the CAM and DNA data
cam = data['cam'].squeeze()  # Ensure CAM is 1D
dna_sequence = data['data']

# Ensure the data is in the correct shape
print('CAM shape:', cam.shape)
print('DNA sequence shape:', dna_sequence.shape)


# Create subplots for CAM
fig, ax2 = plt.subplots(figsize=(12, 4))
ax2.plot(cam)

# Shift the y-axis for the CAM plot
ax2.set_ylim(0.2, ax2.get_ylim()[1])

# Remove y-axis ticks for CAM plot
# ax2.set_yticks([])

# Create subplots for DNA sequence heatmap
fig, ax1 = plt.subplots(figsize=(12, 0.5))
ax1.imshow(dna_sequence, aspect='auto', cmap='Greys_r', norm=plt.cm.colors.PowerNorm(gamma=0.5, vmin=np.min(dna_sequence[dna_sequence > 0]),
                                                                                   vmax=np.max(dna_sequence)))
# ax1.set_yticks([])  # Remove y-axis ticks

# Adjust the layout
plt.tight_layout()

# Show plot with reversed color map for DNA sequence heatmap
plt.show()


In [ ]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec

# Load the data from the .mat file
data = sio.loadmat('new.mat')

# Extract the CAM and DNA data
cam = data['cam'].squeeze()  # Ensure CAM is 1D
dna_sequence = data['data']

# Ensure the data is in the correct shape
print('CAM shape:', cam.shape)
print('DNA sequence shape:', dna_sequence.shape)

# Create a figure
fig = plt.figure(figsize=(12, 4.5))  # Total height is 4.5 to accommodate both plots

# Create a GridSpec with 2 rows and 1 column, set height ratios
gs = GridSpec(2, 1, height_ratios=[4, 0.5], hspace=0.05)

# Create subplot for the CAM plot
ax2 = fig.add_subplot(gs[0])
ax2.plot(cam)
ax2.set_ylim(0.1, ax2.get_ylim()[1])
# ax2.set_ylabel('CAM Intensity', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=12)

# Create subplot for the DNA sequence heatmap
ax1 = fig.add_subplot(gs[1])
ax1.imshow(dna_sequence, aspect='auto', cmap='Greys_r', norm=plt.cm.colors.PowerNorm(gamma=2, vmin=np.min(dna_sequence[dna_sequence > 0]), vmax=np.max(dna_sequence)))
ax1.set_yticks([])  # Remove y-axis ticks
# ax1.set_xlabel('Sequence Position', fontsize=18)
# ax1.set_ylabel('HeatMap', fontsize=18)
ax1.tick_params(axis='both', which='major', labelsize=12)

# Adjust the layout
plt.tight_layout()

# Save the plot as a file
plt.savefig('sparse1.png')

# Show plot
plt.show()


In [ ]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

# Load the data from the .mat file
data = sio.loadmat('new.mat')

# Extract the CAM and DNA data
cam = data['cam'].squeeze()  # Ensure CAM is 1D
dna_sequence = data['data']

# Ensure the data is in the correct shape
print('CAM shape:', cam.shape)
print('DNA sequence shape:', dna_sequence.shape)

# Plot the DNA sequence as a heatmap if it's 2D
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.title('DNA Sequence Heatmap')
if len(dna_sequence.shape) == 2:
    plt.imshow(dna_sequence, aspect='auto', cmap='viridis')
    plt.colorbar(label='Intensity')
    plt.xlabel('Sequence Position')
    plt.ylabel('Feature Index')
else:
    plt.plot(dna_sequence)
    plt.xlabel('Sequence Position')
    plt.ylabel('Intensity')

# Plot the CAM result as a line plot
plt.subplot(1, 2, 2)
plt.title('Class Activation Map (CAM)')
plt.plot(cam)
plt.xlabel('Sequence Position')
plt.ylabel('CAM Intensity')

plt.tight_layout()
plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# 加载 tensor
bdl_tensor = torch.load('1st_gate_promoterall.pt')

# 确认形状
print('Loaded tensor shape:', bdl_tensor.shape)  # (Batch, Depth, Length)

# 将 tensor 转换为 numpy 数组
bdl_np = bdl_tensor.cpu().numpy()

# 展示第一个 batch 的所有 depth 的数据分布
batch_index = 0

# 直方图
plt.figure(figsize=(12, 6))
for depth_index in range(bdl_np.shape[1]):
    plt.subplot(1, bdl_np.shape[1], depth_index + 1)
    sns.histplot(bdl_np[batch_index, depth_index, :], kde=True)
    plt.title(f'Batch {batch_index}, Depth {depth_index}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# 热图
plt.figure(figsize=(12, 6))
for depth_index in range(bdl_np.shape[1]):
    plt.subplot(1, bdl_np.shape[1], depth_index + 1)
    sns.heatmap(bdl_np[batch_index, depth_index, :].reshape(1, -1), cmap='viridis', cbar=True)
    plt.title(f'Batch {batch_index}, Depth {depth_index}')
    plt.xlabel('Sequence Position')
plt.tight_layout()
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 加载 tensor
bdl_tensor = torch.load('1st_gate_promoterall.pt').cpu()

# 确认形状
print('Loaded tensor shape:', bdl_tensor.shape)  # (Batch, Depth, Length)

# 使用 reshape 重新排列数据为二维数组
reshaped_tensor = bdl_tensor.reshape(-1, bdl_tensor.shape[2])  # (Batch*Depth, Length)

# 使用 permute 调整维度顺序
permuted_tensor = bdl_tensor.permute(2, 1, 0)  # (Length, Depth, Batch)

# 将 tensor 转换为 numpy 数组
bdl_np = bdl_tensor.cpu().numpy()
reshaped_np = reshaped_tensor.numpy()
permuted_np = permuted_tensor.numpy()

# 获取 tensor 的形状
batch_size, depth_size, length_size = bdl_np.shape

# 创建 3D 坐标
b, d, l = np.meshgrid(range(batch_size), range(depth_size), range(length_size), indexing='ij')

# 扁平化数据
l = l.flatten()  # Length
d = d.flatten()  # Depth
b = b.flatten()  # Batch
values = bdl_np.flatten()

# 创建 3D 图形
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# 绘制散点图，调整点的大小和透明度
sc = ax.scatter(l, d, b, c=values, cmap='plasma', s=20, alpha=0.6)

# 添加颜色条
cbar = plt.colorbar(sc, ax=ax, label='Value')
cbar.set_ticks([0, 0.25, 0.5, 0.75, 1])
cbar.set_ticklabels(['0', '0.25', '0.5', '0.75', '1'])

# 设置坐标轴标签
ax.set_xlabel('Length')
ax.set_ylabel('Depth')
ax.set_zlabel('Batch')

# 设置标题
plt.title('3D Visualization of BDL Tensor with Enhanced Color Contrast')

plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 加载 tensor
bdl_tensor = torch.load('1st_gate_promoterall.pt')

# 确认形状
print('Loaded tensor shape:', bdl_tensor.shape)  # (Batch, Depth, Length)

# 将 tensor 转换为 numpy 数组
bdl_np = bdl_tensor.cpu().numpy()

# 获取 tensor 的形状
batch_size, depth_size, length_size = bdl_np.shape

# 创建 3D 坐标
b, d, l = np.meshgrid(range(batch_size), range(depth_size), range(length_size), indexing='ij')

# 扁平化数据
l = l.flatten()  # Length
d = d.flatten()  # Depth
b = b.flatten()  # Batch
values = bdl_np.flatten()

# 创建 3D 图形
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# 绘制散点图，调整点的大小和透明度
sc = ax.scatter(b, l, d, c=values, cmap='plasma', s=20, alpha=0.6)

# 添加颜色条
cbar = plt.colorbar(sc, ax=ax, label='Value')
cbar.set_ticks([0, 0.25, 0.5, 0.75, 1])
cbar.set_ticklabels(['0', '0.25', '0.5', '0.75', '1'])

# 设置坐标轴标签
ax.set_xlabel('Batch')
ax.set_ylabel('Length')
ax.set_zlabel('Depth')

# 设置标题
plt.title('3D Visualization of BDL Tensor with Enhanced Color Contrast')

plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 加载 tensor
bdl_tensor = torch.load('1st_gate_promoterall.pt')

# 使用 permute 调整维度顺序为 LBD
lbd_tensor = bdl_tensor.permute(2, 0, 1).cpu()  # (Length, Batch, Depth)

# 将 tensor 转换为 numpy 数组
lbd_np = lbd_tensor.numpy()

# 获取 tensor 的形状
length_size, batch_size, depth_size = lbd_np.shape

# 创建 3D 坐标
l, b, d = np.meshgrid(range(length_size), range(batch_size), range(depth_size), indexing='ij')

# 扁平化数据
l = l.flatten()  # Length
b = b.flatten()  # Batch
d = d.flatten()  # Depth
values = lbd_np.flatten()

# 创建 3D 图形
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# 绘制散点图，调整点的大小和透明度
sc = ax.scatter(l, b, d, c=values, cmap='plasma', s=20, alpha=0.6)

# 添加颜色条
cbar = plt.colorbar(sc, ax=ax, label='Value')
cbar.set_ticks([0, 0.25, 0.5, 0.75, 1])
cbar.set_ticklabels(['0', '0.25', '0.5', '0.75', '1'])

# 设置坐标轴标签
ax.set_xlabel('Length')
ax.set_ylabel('Batch')
ax.set_zlabel('Depth')

# 设置标题
plt.title('3D Visualization of LBD Tensor with Enhanced Color Contrast')

plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# 加载 tensor
bdl_tensor = torch.load('1st_gate_promoterall.pt').cpu()

# 提取 Batch 中的第一个切片
lbd_slice = bdl_tensor[0]

# 将 tensor 转换为 numpy 数组
lbd_slice_np = lbd_slice.numpy()

# 获取 tensor 的形状
depth_size, length_size = lbd_slice_np.shape

# 创建二维网格，注意尺寸+1
l = np.arange(length_size + 1)
d = np.arange(depth_size + 1)
L, D = np.meshgrid(l, d)

# 创建图形
fig, ax = plt.subplots(figsize=(12, 8))

# 绘制二维图像，调整颜色映射和边界
c = ax.pcolormesh(L, D, lbd_slice_np, cmap='viridis', shading='auto')

# 添加颜色条
fig.colorbar(c, ax=ax, label='Value')

# 设置坐标轴标签
ax.set_xlabel('Length')
ax.set_ylabel('Depth')

# 设置标题
plt.title('2D Visualization of LBD Tensor Slice (First Batch)')

plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# 加载 tensor
bdl_tensor = torch.load('1st_gate_promoterall.pt').cpu()

# 提取 Batch 中的第一个切片
lbd_slice = bdl_tensor[1]

# 将 tensor 转换为 numpy 数组
lbd_slice_np = lbd_slice.numpy()

# 获取 tensor 的形状
depth_size, length_size = lbd_slice_np.shape

# 创建二维网格，注意尺寸+1
l = np.arange(length_size + 1)
d = np.arange(depth_size + 1)
L, D = np.meshgrid(l, d)

# 创建图形
fig, ax = plt.subplots(figsize=(12, 8))

# 绘制二维图像，调整颜色映射和边界
c = ax.pcolormesh(L, D, lbd_slice_np, cmap='viridis', shading='auto')

# 添加颜色条
fig.colorbar(c, ax=ax, label='Value')

# 设置坐标轴标签
ax.set_xlabel('Length')
ax.set_ylabel('Depth')

# 设置标题
plt.title('2D Visualization of LBD Tensor Slice (First Batch)')

plt.show()
